In [1]:
import numpy as np
import csv
import pandas as pd

In [2]:
# dedicated runtime
appRuntime = np.load('../dedicated_runtime/app_dedicated_rt.npy').item()

In [3]:
def read_timing_df(curfile):
    """read timing traces in csv file, offset the trace with the starting pos, return the dataframe"""
    df_current = pd.read_csv(curfile)
    offset = df_current['start'].min()  # find the starting timing, and offset it
    df_current['start'] = df_current['start'].apply(lambda x: x - offset)
    df_current['end'] = df_current['end'].apply(lambda x: x - offset)
    return df_current

def compare_with_dedicatedruntime(df, appRuntime):
    """
    read the timing traces in dataframe (after read_timing_df()).
    compute the speedup over the dedicated runtime
    return speedup list
    """
    dedicated_total,  corun_total = 0.0, 0.0
    speedup_list = []

    # columns => ['jobID', 'appName', 'start', 'end', 'duration (s)']
    for index, row in df.iterrows():
        app_name, app_endT = row['appName'], row['end']
        app_rt_dedicate = float(appRuntime[app_name])  # dedicated runtime
        app_endT = float(app_endT)
        if corun_total < app_endT: corun_total = app_endT
        dedicated_total += app_rt_dedicate
        speedup = dedicated_total / corun_total
        speedup_list.append(speedup)   
    return speedup_list

In [4]:
len(appRuntime)

79

In [5]:
appRuntime

{'cudasdk_BlackScholes': '1.4004418849945068',
 'cudasdk_FDTD3d': '14.441001176834106',
 'cudasdk_MCEstimatePiInlineP': '0.39530420303344727',
 'cudasdk_MCEstimatePiInlineQ': '0.3933548927307129',
 'cudasdk_MCEstimatePiP': '0.3939080238342285',
 'cudasdk_MCEstimatePiQ': '0.4072442054748535',
 'cudasdk_MCSingleAsianOptionP': '0.39748215675354004',
 'cudasdk_SobolQRNG': '0.5573899745941162',
 'cudasdk_batchCUBLAS': '0.7025609016418457',
 'cudasdk_binomialOptions': '25.258795976638794',
 'cudasdk_boxFilterNPP': '0.4159379005432129',
 'cudasdk_c++11Cuda': '0.5548751354217529',
 'cudasdk_concurrentKernels': '0.39653706550598145',
 'cudasdk_convolutionFFT2D': '3.801888942718506',
 'cudasdk_convolutionSeparable': '1.9495930671691895',
 'cudasdk_convolutionTexture': '1.1159520149230957',
 'cudasdk_dct8x8': '0.7084078788757324',
 'cudasdk_dwtHaar1D': '0.39554309844970703',
 'cudasdk_dxtc': '0.41524600982666016',
 'cudasdk_eigenvalues': '1.3545660972595215',
 'cudasdk_fastWalshTransform': '3.368

In [6]:
def compute_antt(inputcsv):
    """Read app status table in csv format, compute the ANTT"""
    
    df = read_timing_df(inputcsv)
    
    #
    # 1) sort the dataframe by 'end'
    #
    sorted_df = df.sort_values(by=['end'])
    
    
    #
    # 2) compute the ntt for each app
    # 
    antt_list = []
    for idx, row in sorted_df.iterrows():
        appName = row['appName']
        T_corun = float(row['duration (s)'])
        T_dedicate = float(appRuntime[appName])

        tmp = T_corun / T_dedicate
        #print appName, T_corun, T_dedicate, tmp
        antt_list.append(tmp)

    #
    # 3) in antt list, add current value with prev
    # 
    antt = [0 for i in xrange(len(antt_list))]
    for i, v in enumerate(antt_list):
        if i == 0:
            antt[0] = v
        else:
            antt[i] = antt[i-1] + v

    # 4) final step:  1 / N on each data sample
    antt_seq = []
    for i, v in enumerate(antt):
        #print i, v
        N = i + 1
        current_antt = v / N
        #print current_antt
        antt_seq.append(current_antt)
    
    return antt_seq
    

### read traces

In [7]:
# we have featall/feat64/feat42/feat42/feat26/feat18/feat14/feat12/featMystic, 
# to compare with fcfs

In [8]:
inputcsv = "../fcfs_csv/fcfs_run8_s1.csv"
antt = compute_antt(inputcsv)
print antt

mydf = pd.DataFrame(antt)
mydf.to_csv('antt_fcfs_max8.csv', index=False, header=False)

[8.141154894735056, 7.882952437431185, 6.661049971908845, 6.5339365058419645, 6.252745418748438, 5.778596859623275, 5.6741020374531805, 5.256332457247241, 4.9507685740946386, 4.894434914965556, 4.884116452250333, 4.730610973835241, 4.742692167724997, 4.825419433258255, 4.864678000814529, 4.71922148173679, 4.695724901235579, 4.523514674717192, 4.405975205975314, 4.337227495772011, 4.219856665052021, 4.170138106795753, 4.1137216731354815, 4.077238402746715, 4.020228418937237, 3.9440065973783405, 3.917117193790588, 3.8399645200196986, 3.8353490483037485, 3.7691632981425154, 3.7323196410176167, 3.672889177564875, 3.7156915644445894, 3.6649053489003154, 3.663026568227586, 3.643430121273992, 3.60746594708942, 3.5935860616632946, 3.56717120939441, 3.5299386751525375, 3.5172400746130514, 3.509700340336279, 3.51665928223064, 3.472679054886505, 3.467587470102034, 3.4990012010319163, 3.5051614903456905, 3.465302618663347, 3.4824735803159266, 3.472876859381768, 3.482721946199762, 3.474661943448571

In [9]:
inputcsv = "../sim_csv/feat9/sim_feat9_run8_s1.csv"
antt = compute_antt(inputcsv)
print antt

mydf = pd.DataFrame(antt)
mydf.to_csv('antt_sim-feat9_max8.csv', index=False, header=False)

[7.572090858799185, 8.612610994291122, 7.3455725534527545, 6.4463856135537, 6.117673425871414, 5.957420604301936, 5.470523621815865, 5.164255243434283, 4.911174625239443, 4.73651479800139, 4.651612223891669, 4.550922018975366, 4.4547657051618526, 4.389569040132886, 4.268831577040833, 4.392401354459975, 4.343642044520682, 4.333446196970026, 4.566423282882051, 4.725590189046253, 4.640384817928831, 4.622651405810379, 4.637557995331275, 4.607387993786927, 4.604861954611801, 4.612951941563611, 4.6174491584478226, 4.694004765363059, 4.696559180957679, 4.802250493640958, 4.843251179824461, 4.86716716179291, 4.861804273295409, 4.825987989077137, 4.825378062377467, 4.830898559600295, 4.823432374071692, 4.815447150133891, 4.80602592670205, 4.75389154443848, 4.7199090237019465, 4.670457006061464, 4.649547212267588, 4.633222325277424, 4.595179220064484, 4.572560903320325, 4.532280228640979, 4.518674516117048, 4.484931752326467, 4.4485961451919005, 4.41282884309579, 4.39056888556065, 4.374315100342

In [10]:
inputcsv = "../robustBin_csv/feat9/robustBin_feat9_run8_s1.csv"
antt = compute_antt(inputcsv)
print antt

mydf = pd.DataFrame(antt)
mydf.to_csv('antt_interBin-feat9_max8.csv', index=False, header=False)

[4.994248583439956, 4.597246078828775, 4.061260868498736, 3.601930316256854, 3.475262768550666, 3.193303260754426, 3.1753339478402856, 3.0042242696870796, 3.061641683028978, 2.902699403451201, 2.9183927585247402, 2.8868492219973465, 2.8882861094864976, 2.837126637344005, 2.8181860578617672, 2.8224230431857618, 2.810536856763651, 2.8321548303173967, 2.858374108553214, 2.8558200088951233, 2.8833748185913697, 2.8671128947062487, 2.8737118266538237, 2.858510195162727, 2.8213215486458294, 2.832420863917714, 2.788584145069492, 2.791535435542982, 2.792867734482657, 2.756284036439776, 2.7404946416859657, 2.744122791465571, 2.7446816709260946, 2.7483738003297247, 2.7274250931021435, 2.6958316470232746, 2.6850275570182975, 2.6776826831783436, 2.673500844507686, 2.666014316556378, 2.654474214497479, 2.622729771985078, 2.605254528264429, 2.588069726496019, 2.593122907189252, 2.6132644433985366, 2.6368362726886105, 2.6671128712977805, 2.6502720850784423, 2.627807492835572, 2.6295210557667112, 2.612